In [19]:
import pandas as pd
import numpy as np

In [20]:
# loading data
df = pd.read_csv("train.csv")

In [4]:
# greet the data
print(df.describe())
print("-"*65)
print(df.describe(include=['O']))
print("-"*65)
print(df.info())

                  Id    Population         Weight    TargetValue
count  844972.000000  8.449720e+05  844972.000000  844972.000000
mean   484802.500000  2.720127e+06       0.530870      10.959682
std    279911.120335  3.477771e+07       0.451909     275.699242
min         1.000000  8.600000e+01       0.047491  -10034.000000
25%    242383.750000  1.213300e+04       0.096838       0.000000
50%    484802.500000  3.053100e+04       0.349413       0.000000
75%    727221.250000  1.056120e+05       0.968379       0.000000
max    969604.000000  1.395773e+09       2.239186   36163.000000
-----------------------------------------------------------------
            County Province_State Country_Region        Date          Target
count       766892         799344         844972      844972          844972
unique        1840            133            187         122               2
top     Washington          Texas             US  2020-05-16  ConfirmedCases
freq          7564          62220        

In [21]:
# create training and testing datasets and actual labels
train = df.copy()
train = train[train.Date <= '2020-05-10']

test = df.copy()
test = test[(test['Date']>='2020-05-11')&(test['Date']<='2020-05-17')]

actuals = test.TargetValue

In [20]:
'''# seems like the data is cleaned
train.Country_Region.unique().tolist()
train.County.unique().tolist()
train.Province_State.unique().tolist()'''

'# seems like the data is cleaned\ntrain.Country_Region.unique().tolist()\ntrain.County.unique().tolist()\ntrain.Province_State.unique().tolist()'

In [32]:
'''# See the training data
print(train.describe())
print("-"*65)
print(train.describe(include=['O']))
print("-"*65)
print(train.info())'''

'# See the training data\nprint(train.describe())\nprint("-"*65)\nprint(train.describe(include=[\'O\']))\nprint("-"*65)\nprint(train.info())'

A couple of ideas for models:
- quntile linear regression from mlinsights
- quantile gradient boosting regressor from sklearn
- quantile regression forests from skargen
- quantile deep net using Keras or Tensorflow

A couple of ideas for the missing values:
- drop 
- impute
- impute with mark on which ones are missing

A couple of ideas for the columns:

- County: drop or keep
- Province_State: drop or keep
- Population: bins or as is
- Date: day of week, day of month, month, day of year

- relationship b/w confirmed and death?

- New columns:Continuent ?

Ways of encoding

...

In [22]:
#Create the Weighted Pinball Loss function

def PLoss(preds, actuals, qt):
    loss = np.maximum((actuals-preds)*qt, (preds-actuals)*(1-qt))
    return loss

def WPLoss(preds05, preds50, preds95, actuals, weights):
    l05 = PLoss(preds05, actuals,0.05)
    l50 = PLoss(preds50, actuals,0.5)
    l95 = PLoss(preds95, actuals,0.95)
    score = sum((l05+l50+l95)/3*weights)/actuals.count()
    return score

In [23]:
# Create label y
y = train.TargetValue

# Clean train and test data
train = train.drop(['Id','TargetValue'],axis=1)
testId = test.Id
test = test.drop(['Id','TargetValue'],axis=1)

In [24]:
#date

train.Date = pd.to_datetime(train.Date)
test.Date = pd.to_datetime(test.Date)

for ds in [train, test]:
  ds['Day'] = ds.Date.dt.day
  ds['Month'] = ds.Date.dt.month
  ds['Dayofweek'] = ds.Date.dt.dayofweek
  ds['Dayofyear'] = ds.Date.dt.dayofyear
  ds = ds.drop(['Date'],axis=1)

In [25]:
# drop columns with missing values for now
train_noMis = train.copy()
test_noMis = test.copy()

train_noMis = train_noMis.drop(['County','Province_State','Date'],axis=1)
test_noMis = test_noMis.drop(['County','Province_State','Date'],axis=1)

In [27]:
# Split train data into validation and training data
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(train_noMis, y,random_state=1, test_size=0.25)

Let's try the models with droping the columns with missing values


In [28]:
# encode Country_Region and Target
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_Xen = train_X.copy()
val_Xen = val_X.copy()

for col in ['Country_Region','Target']:
  train_Xen[col] = label_encoder.fit_transform(train_X[col])
  val_Xen[col] = label_encoder.transform(val_X[col])

In [29]:
# Not sure what the 'quantile' loss function in GBR!

#GradientBoostingRegressor

from sklearn.ensemble import GradientBoostingRegressor

def gbrPred(train_X, val_X, train_y,val_y,qt):
    model = GradientBoostingRegressor(loss='quantile', alpha=qt)
    model.fit(train_X,train_y)
    
    preds = model.predict(val_X)
    return preds

In [ ]:

gbr05 = gbrPred(train_Xen, val_Xen, train_y,val_y,0.05)
gbr50 = gbrPred(train_Xen, val_Xen, train_y,val_y,0.5)
gbr95 = gbrPred(train_Xen, val_Xen, train_y,val_y,0.95)

gbr_score = WPLoss(gbr05, gbr50, gbr95, val_y, val_X.Weight)

In [30]:
gbr05 = gbrPred(train_Xen, val_Xen, train_y,val_y,0.05)

In [31]:
gbr50 = gbrPred(train_Xen, val_Xen, train_y,val_y,0.5)

In [32]:
gbr95 = gbrPred(train_Xen, val_Xen, train_y,val_y,0.95)

In [33]:
gbr_score = WPLoss(gbr05, gbr50, gbr95, val_y, val_X.Weight)

In [35]:
print('gbr05:')
print(gbr05)
print('gbr50:')
print(gbr50)
print('gbr95:')
print(gbr95)
print('gbr weighted score:')
print(gbr_score)

gbr05:
[0. 0. 0. ... 0. 0. 0.]
gbr50:
[0.   0.   0.   ... 0.   0.   0.19]
gbr95:
[1.77795138e-01 8.33825906e-01 4.65102475e+00 ... 3.23508112e+02
 6.82135758e-01 2.71198918e+00]
gbr weighted score:
0.2999235618524082


In [ ]:
# quantile linear regression
#from mlinsights.mlmodel import QuantileLinearRegression

In [ ]:
# Deep nets
